# re:Invent 2019 Amazon Personalize 워크숍

![MainDiagram](static/imgs/image.png)

## 진행 순서

Amazon Personalize의 작업 프로세스를 크게 보면 아래 다이어그램의 단계에 따라 진행됩니다.

![FlowDiagram](static/imgs/personalize_process.png)

따르게 되는 구체적인 프로세스는 다음과 같습니다.

1. 가져오기 및 설정
1. 데이터 준비
1. 데이터 가져오기
1. 레시피 선택
1. 솔루션 훈련
1. 캠페인 배포
1. 추천 받기
1. 실시간 상호 작용
1. 결론
1. 보너스: 추천의 대량 내보내기

이러한 각 단계에서 Boto3 SDK를 사용하여 Python으로 작성된 코드 조각을 보고 실행할 수 있습니다. 이러한 코드 조각은 Personalize와의 프로덕션 통합의 구성 요소가 되도록 수정할 수 있습니다.

이 노트북에서는 특정 사용자에 맞춤화된 영화 추천 모델을 구축하는 단계를 안내합니다. 여기서 목표는 영화와 긍정적으로 상호 작용한 사용자의 이력을 기반으로 영화를 추천하는 것입니다.

이 데이터는 [MovieLens 프로젝트](https://movielens.org)를 통해 제공되며, 관심 있는 경우 나중에 자세히 읽어볼 수 있습니다.

아래의 콘텐츠는 워크숍 세션을 진행하는 동안 프로세스를 안내할 목적으로 작성되었습니다. 나중에 GitHub에서 찾은 CloudFormation 템플릿을 본인의 계정에 배포하고, 노트북 1~3에서 동일한 연습을 다시 수행할 수 있습니다. 또한 셀의 콘텐츠를 업데이트하여 자체 데이터를 반영할 수 있으며, 이는 사용 사례에 따라 사용자 지정 추천 모델을 구축하는 데 효과적인 방법입니다.

## 이 노트북을 사용하는 방법

코드는 아래와 같이 셀로 구분됩니다. 이 페이지 맨 위에 삼각형 모양의 `Run` 버튼이 있습니다. 이 버튼을 클릭하여 각 셀을 실행하고 다음 셀로 이동하거나, 셀에 있는 동안 `Shift` + `Enter` 또는 `Shift` + `Return`(Mac 사용자)을 눌러 셀을 실행한 후 다음 셀로 이동할 수 있습니다.


셀이 실행되면, 셀이 실행되는 동안에는 옆의 라인에 `*`가 표시되며, 셀 내의 모든 코드 실행을 완료하고 나면 이 기호가 실행을 완료한 마지막 셀을 나타내는 숫자로 업데이트됩니다.


아래 지침에 따라 셀을 실행하여 Amazon Personalize를 시작하세요.



## 가져오기 및 설정 

Python은 다양한 라이브러리 컬렉션과 함께 제공되며, 핵심 데이터 과학 도구로서 설치된 boto3(AWS SDK), Pandas/Numpy 등을 라이브러리와 함께 가져와야 합니다. 아래의 셀은 여기서 사용할 수 있도록 도구를 가져옵니다. 또한 boto3 라이브러리를 최신 버전으로 업데이트합니다. 경고(노란색 텍스트) 또는 오류(빨간색 텍스트)가 표시될 수 있습니다. 이는 정상적인 동작이므로 다음 셀을 계속 실행하면 됩니다.

In [ ]:
# Update boto3
!pip install --upgrade boto3
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
import datetime
import uuid

그런 다음 사용자 환경이 Amazon Personalize와 성공적으로 통신할 수 있는지 검증해야 합니다. 아래 코드 줄은 이 작업을 수행합니다.

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')
personalize_events = boto3.client(service_name='personalize-events')

설정의 마지막 부분에서는 이 워크숍을 실행하는 리전을 결정합니다. 아래의 셀은 이 작업을 수행하고 그 결과를 `region` 변수에 할당합니다.

In [ ]:
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]

## 데이터 준비

시작하려면 사용자가 어떤 식으로든 콘텐츠와 상호 작용한 데이터 포인트의 컬렉션이 필요합니다. Amazon Personalize는 상호 작용이 기록된 경우 긍정적인 상호 작용으로 간주합니다. 상호 작용이 어떻게 사용되는지는 나중에 살펴보겠습니다.

아래의 셀은 필요한 데이터를 다운로드하고 Zip 파일에서 콘텐츠를 추출한 다음, 그중 일부를 화면에 표시합니다.

In [ ]:
!wget -N http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -o ml-100k.zip
data = pd.read_csv('./ml-100k/u.data', sep='\t', names=['USER_ID', 'ITEM_ID', 'RATING', 'TIMESTAMP'], engine='python')
pd.set_option('display.max_rows', 5)
data

보시다시피, 이 데이터에는 UserID, ItemID, 평점 및 타임스탬프가 포함됩니다.

`Rating` 열은 필요 없으므로 파일을 저장하기 전에 이 열이 제거됩니다.

데이터가 준비되면 마지막 코드 줄을 사용하여 로컬 CSV 파일에 데이터를 저장합니다.

In [ ]:
data = data[['USER_ID', 'ITEM_ID', 'TIMESTAMP']] # select columns that match the columns in the schema below
filename = "movie-lens-ml-100k-prepared.csv"
data.to_csv(filename, index=False)

데이터를 S3에 업로드하려면 먼저 버킷을 생성해야 합니다. 아래의 셀은 이 작업을 수행합니다.

In [ ]:
print(region)
s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
bucket_name = account_id + "reinventpersonalizeworkshop"
print(bucket_name)
if region != "us-east-1":
    s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region})
else:
    s3.create_bucket(Bucket=bucket_name)

이제 S3에 파일을 업로드하면, Amazon Personalize로 가져올 준비가 끝납니다.

In [ ]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(filename).upload_file(filename)

## 데이터 가져오기

데이터를 가져오는 단계는 다음과 같습니다.

1. 데이터 세트 그룹을 생성합니다.
1. 데이터 세트의 스키마를 결정합니다.
1. 스키마를 사용하여 데이터 세트를 만듭니다.
1. 가져오기 작업을 실행하여 Personalize에 사용할 데이터를 로드합니다.


Amazon Personalize에서 데이터 세트 그룹은 실험 간에 정보를 격리하는 수단입니다. 이러한 그룹 간에는 정보가 전혀 공유되지 않습니다. 데이터 세트 그룹에는 상호 작용 데이터, 항목 메타데이터, 사용자 메타데이터, 이벤트 트래커, 솔루션 및 캠페인이 포함될 수 있습니다. 데이터 세트 그룹에 대한 자세한 내용은 https://docs.aws.amazon.com/personalize/latest/dg/API_DatasetGroup.html 을 참조하세요.

데이터 세트 그룹을 만드는 것부터 시작하기

#### 데이터 세트 그룹 생성

In [ ]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "personalize-RI-demo"
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

#### 데이터 세트 그룹 생성 루프

DatasetGroup을 만드는 데 몇 초 정도 걸리므로, 아래의 셀은 DatasetGroup이 활성화되고 워크숍을 계속 진행할 수 있게 될 때까지 폴링합니다. `ACTIVE`로 표시되면 `Create Schema` 단계로 넘어갑니다.


In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(5)

#### 스키마 생성


이제 데이터 세트의 스키마를 결정할 수 있습니다. 이 워크숍에서는 시간이 제한되어 있으므로, 사용자-항목-상호 작용 데이터 또는 줄여서 상호 작용 데이터만 활용합니다. 아래의 셀에는 이 데이터 세트에 필요한 항목이 포함되어 있으며, 이전에 업로드한 CSV의 구조에 매핑됩니다.

In [ ]:
schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

아래의 셀은 데이터 세트에 연결할 수 있는 스키마를 Amazon Personalize 내부에 생성합니다. 이렇게 하면 Personalize가 CSV 파일의 콘텐츠를 이해할 수 있습니다.

In [ ]:
create_schema_response = personalize.create_schema(
    name = "personalize-ri-demo-schema",
    schema = json.dumps(schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

#### 데이터 세트 생성


다음으로, 상호 작용에 대한 데이터 세트를 만들고 제공된 스키마를 할당한 후 다음 코드를 사용하여 이전에 만든 데이터 세트 그룹에 할당합니다.

In [ ]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-ri-interactions",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn
)

dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

#### S3 버킷에 정책 연결

데이터에 대한 가져오기 작업을 실행하려면 먼저 S3 버킷에 버킷 정책을 연결하여 Personalize가 S3 버킷과 통신할 수 있도록 하고 이 AWS 계정에서 사용할 서비스에 대한 IAM 역할을 부여해야 합니다.

In [ ]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:*Object",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

참고로 아래의 셀에서 변경 내용을 확인하는 데 몇 초밖에 걸리지 않으므로 이 섹션은 1분이면 완료됩니다. 다음으로 아래의 셀을 실행합니다.

#### Personalize 역할 생성

In [ ]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleRIDemo"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

이제 데이터를 Personalize로 가져올 준비가 되었습니다. 아래의 첫 번째 셀은 그 프로세스를 시작하고, 두 번째 셀에는 서비스를 폴링하여 가져오기 작업이 언제 완료되었는지 확인하는 루프가 포함되어 있습니다.

이 워크숍에서는 비교적 작은 데이터 세트를 가져오는데, 작은 파일인데도 시간이 많이 걸린다고 느껴질 수 있습니다. 여기서 시간이 가장 많이 소요되는 부분은 실제로 작업을 실행할 전용 리소스를 프로비저닝하는 것입니다. 이를 통해 Personalize는 HIPAA 규정 준수 등을 제공할 수 있습니다. 즉, 파일이 더 크다고 해서 리소스가 활성화되어 가져오기를 실행하는 시간이 엄청나게 오래 걸리지는 않습니다.

가져오기 프로세스가 완료되는 데 최대 20분이 걸릴 수 있습니다. 

#### 데이터 세트 가져오기 작업 생성

In [ ]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-ri-import",
    datasetArn = dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

#### 데이터 세트 가져오기 루프

아래의 셀을 완료하는 데는 시간이 좀 더 걸립니다. 데이터 세트를 Personalize로 완전히 가져온 시점을 알아보기 위해 폴링를 실행합니다. 이 작업은 완료하는 데 20분 정도 걸립니다. 대부분의 시간은 내부적으로 인프라를 프로비저닝하는 데 소요됩니다. 즉, 대부분의 경우 훨씬 큰 파일을 가져오더라도 시간이 훨씬 오래 걸리지는 않습니다.

아래의 셀을 실행한 후 `ACTIVE` 상태가 되면 계속 진행합니다.


In [ ]:
current_time = datetime.datetime.now()
print("Import Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.datetime.now()
print("Import Completed on: ", current_time.strftime("%I:%M:%S %p"))

## 레시피 선택

Personalize에서 사용 가능한 알고리즘을 레시피라고 하며, 아래의 코드는 전체 레시피 목록을 제공합니다. 각 레시피마다 다양한 사용 사례가 있으며 자세한 내용은 https://docs.aws.amazon.com/personalize/latest/dg/working-with-predefined-recipes.html 에서 확인할 수 있습니다.

이 워크숍에서는 HRNN(Hierarchical Recurrent Neural Network)을 사용하여 다음 문서를 인용합니다.

```
HRN은 계층적 반복 신경망으로, 일정 시간에 걸쳐 사용자-항목 상호 작용을 모델링할 수 있습니다. 시간이 지남에 따라 사용자 행동이 변화하는 경우 HRNN 레시피를 사용합니다. 이 레시피는 진화하는 의도 문제를 나타냅니다

HRNN은 데이터 세트 그룹의 상호 작용 데이터 세트를 사용하여 모델을 훈련합니다. 데이터 세트 그룹은 사용자, 항목 및 상호 작용 데이터 세트를 포함할 수 있는 일련의 관련 데이터 세트입니다.
```

추천과 관련하여 데이터 세트 그룹에 대해 자세히 설명하는 문서를 https://openreview.net/pdf?id=ByzxsrrkJ4 에서 참조할 수 있습니다.


여기서는 심층 신경망을 사용하여 구축된 추천 시스템의 예로 사용되었으며, 상호 작용 데이터만 사용하여 훈련할 수 있습니다. 나중에 직접 실험을 실행할 때도 여기서부터 시작할 수 있습니다.

In [ ]:
personalize.list_recipes()

아래의 셀은 `HRNN` 레시피를 선택합니다.

In [ ]:
recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn"

## 솔루션 훈련

Amazon Personalize에서 고객 데이터에 대해 훈련된 모델을 솔루션이라고 합니다. 솔루션은 버전으로 관리되며, 먼저 솔루션을 만든 후 솔루션 버전을 만듭니다. 버전은 사용 가능한 최신 데이터를 기반으로 시간 경과에 따른 모델의 개선 상태를 추적하는 데 사용됩니다.

솔루션을 만드는 작업 자체는 거의 즉각적으로 실행되지만 실제 훈련을 거쳐 버전을 생성하는 데에는 다소 시간이 걸릴 수 있습니다. 일반적으로 이 프로세스에서는 이때가 가장 긴 대기 시간입니다. 워크숍 외부에서 이 작업을 수행할 경우, 이메일을 확인하거나 커피를 마시는 등의 시간으로 활용하면 됩니다.

#### 솔루션 생성

In [ ]:
create_solution_response = personalize.create_solution(
    name = "personalize-ri-soln-hrnn",
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

#### 솔루션 버전 생성

In [ ]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

#### 솔루션 버전 루프 생성

모델을 훈련하는 데 다소 시간이 걸릴 수 있으며, 대개 이 작업을 완료하는 데 20분 이상 소요됩니다. 아래의 셀을 실행하고 다음 단계로 넘어가기 전에 `ACTIVE` 상태가 될 때까지 또 기다립니다.

In [ ]:
current_time = datetime.datetime.now()
print("Training Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.datetime.now()
print("Training Completed on: ", current_time.strftime("%I:%M:%S %p"))

#### 솔루션 버전의 지표 가져오기

이제 솔루션과 버전이 존재하므로 성능을 판단할 지표를 얻을 수 있습니다.

In [ ]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

## 캠페인 생성 및 기다리기

이제 잘 작동하는 솔루션 버전이 있으므로 애플리케이션에 사용할 캠페인을 만들어야 합니다. 캠페인은 모델의 호스팅된 복사본일 뿐입니다. 여기서도 잠시 기다려야 하므로, 실행 후 인프라를 프로비저닝하는 동안 잠시 휴식을 취할 수 있습니다.

#### 캠페인 생성

In [ ]:
create_campaign_response = personalize.create_campaign(
    name = "personalize-ri-camp",
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 1
)

campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

#### 캠페인이 활성 상태가 될 때까지 기다리기

#### 캠페인 루프 생성

이 섹션에서는 Personalize가 모델을 배포하는 데 몇 분 정도 걸립니다. 아래의 셀을 실행하여 작업이 완료될 때까지 다시 한번 폴링합니다. 10~15분 정도 걸립니다. 셀이 `ACTIVE` 상태에 도달하면 계속 진행합니다.

In [ ]:
current_time = datetime.datetime.now()
print("Deploying Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.datetime.now()
print("Deploying Completed on: ", current_time.strftime("%I:%M:%S %p"))

## 샘플 추천 받기

캠페인이 활성 상태가 되면 추천을 받을 준비가 된 것입니다. 먼저 컬렉션에서 임의의 사용자를 선택해야 합니다. 그런 다음 ID뿐만 아니라, 추천을 위한 다른 영화 정보도 얻을 수 있도록 몇 가지 헬퍼 함수를 만듭니다.

In [ ]:
# Getting a random user:
user_id, item_id, _ = data.sample().values[0]
print("USER: {}".format(user_id))

In [ ]:
items = pd.read_csv('./ml-100k/u.item', sep='|', usecols=[0,1], encoding='latin-1', names=['ITEM_ID', 'TITLE'], index_col='ITEM_ID')

def get_movie_title(movie_id):
    """
    Takes in an ID, returns a title
    """
    movie_id = int(movie_id)-1
    return items.iloc[movie_id]['TITLE']

#### GetRecommendations 호출

아래의 코드는 선택한 임의의 사용자에 대한 추천을 받은 다음 데이터 프레임에 추천을 넣어 렌더링합니다.

In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = str(user_id),
)
# Update DF rendering
pd.set_option('display.max_rows', 30)

print("Recommendations for user: ", user_id)

item_list = get_recommendations_response['itemList']

recommendation_list = []

for item in item_list:
    title = get_movie_title(item['itemId'])
    recommendation_list.append(title)
    
recommendations_df = pd.DataFrame(recommendation_list, columns = ['Original Recommendations'])
recommendations_df

## 이벤트 트래커 생성

추천 시스템이 실시간 이벤트에 응답하려면, 이벤트 트래커가 필요합니다. 아래의 코드는 이벤트 트래커를 생성합니다. 해당 이벤트 트래커를 이 실습에서 계속 사용할 수 있습니다. 보다 유용한 이름으로 자유롭게 바꾸세요.

In [ ]:
response = personalize.create_event_tracker(
    name='MovieClickTrackerRI',
    datasetGroupArn=dataset_group_arn
)
print(response['eventTrackerArn'])
print(response['trackingId'])
TRACKING_ID = response['trackingId']

In [ ]:
event_tracker_arn = response['eventTrackerArn']

## 사용자 행동 시뮬레이션

아래의 코드 줄은 특정 항목과 상호 작용하는 사용자를 시뮬레이션하는 코드 샘플을 제공하며, 여기서는 시작할 때와 다른 추천을 받게 됩니다.

In [ ]:
session_dict = {}

In [ ]:
def send_movie_click(USER_ID, ITEM_ID):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[USER_ID]
    except:
        session_dict[USER_ID] = str(uuid.uuid1())
        session_ID = session_dict[USER_ID]
        
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    }
    event_json = json.dumps(event)
        
    # Make Call
    personalize_events.put_events(
    trackingId = TRACKING_ID,
    userId= USER_ID,
    sessionId = session_ID,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': 'EVENT_TYPE',
        'properties': event_json
        }]
    )

def get_new_recommendations_df(recommendations_df, movie_ID):
    # Get the title of the movie for the header of the column
    movie_title_clicked = get_movie_title(movie_to_click)
    # Interact with the movie
    send_movie_click(USER_ID=str(user_id), ITEM_ID=movie_to_click)
    # Sleep for 2 seconds
    time.sleep(2)
    # Get new recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        title = get_movie_title(item['itemId'])
        recommendation_list.append(title)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_title_clicked])
    # Add this dataframe to the old one
    recommendations_df = recommendations_df.join(new_rec_DF)
    return recommendations_df

아래에 있는 3개의 셀은 데이터 세트에 영화를 추가한 다음 각 상호 작용 후의 결과를 렌더링하는 시뮬레이션을 수행합니다. 첫 번째 열은 원래 추천이고, 그 다음에 있는 열들은 사용자가 현재 상호 작용한 각 영화에 대한 열입니다. 열 헤더는 사용자가 상호 작용한 영화의 이름이며, 추천이 어떻계 바뀌는지를 확인할 수 있습니다.

In [ ]:
movie_to_click = 180
recommendations_df = get_new_recommendations_df(recommendations_df, movie_to_click)
recommendations_df

In [ ]:
movie_to_click = 210
recommendations_df = get_new_recommendations_df(recommendations_df, movie_to_click)
recommendations_df

In [ ]:
movie_to_click = 415
recommendations_df = get_new_recommendations_df(recommendations_df, movie_to_click)
recommendations_df

## 결론

이 워크숍에서는 데이터 세트 그룹과 상호 작용을 기반으로 한 데이터 세트를 만들고, 데이터를 기반으로 추천 모델을 훈련하고, 추천을 생성하는 캠페인을 배포하고, 초기 추천을 평가하고, 추천을 더욱 개선하기 위해 실시간 이벤트 추적을 활용해 보았습니다. 아래에는 추천을 내보내는 방법을 보여주는 보너스 자료입니다.

이 콘텐츠는 GitHub에서 공개되며, 여러분의 조직 내에서 맞춤형 추천 모델을 구축하는 데 사용할 수 있습니다. 앞으로 다른 다양한 노트북도 살펴보세요.

감사합니다. 앞으로 기계 학습 프로젝트를 잘 진행하시기 바랍니다.

보너스 자료:

## 배치 추천

지금까지 API 호출을 통해 추천을 생성하고 이벤트 트래커와 상호 작용하는 방법을 살펴보았습니다. 그 방법은 대부분의 애플리케이션에서는 잘 작동하지만, 사용자를 위한 추천을 모두 로컬로 캐싱하거나, 새로운 아이디어를 얻기 위해 추천을 연구해야 하는 경우도 있습니다. 이를 지원하기 위해 Amazon Personalize는 추천을 파일로 한 번에 모두 내보내는 배치 내보내기를 지원합니다. 아래의 셀은 S3에 있는 파일로 추천을 전송하는 과정을 안내한 다음 해당 콘텐츠를 표시합니다. 이 파일은 S3에서 로컬 컴퓨터로 다운로드할 수도 있습니다.

먼저 사용자 ID의 JSON 파일을 생성해야 합니다.

In [ ]:
user_IDs = ['561', '233', '579']

json_input_filename = "json_input.json"
with open(json_input_filename, 'w') as json_input:
    for user_id in user_IDs:
        json_input.write('{"userId": "' + user_id + '"}\n')

이제 이 파일을 S3에 업로드합니다.

In [ ]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(json_input_filename).upload_file(json_input_filename)
s3_input_path = "s3://" + bucket_name + "/" + json_input_filename
print(s3_input_path)

이제 S3에서 입력 파일을 사용할 수 있으므로, 출력 위치를 정의하고 배치 추론 작업을 생성해야 합니다.

In [ ]:
s3_output_path = "s3://" + bucket_name + "/"
print(s3_output_path)

아래의 셀은 해당 배치 작업을 생성합니다.

In [ ]:
personalize_rec = boto3.client(service_name='personalize')
batchInferenceJobArn = personalize_rec.create_batch_inference_job (
    solutionVersionArn = solution_version_arn,
    jobName = "RI-Workshop-Batch-Inference-Job",
    roleArn = role_arn,
    jobInput = 
     {"s3DataSource": {"path": s3_input_path}},
    jobOutput = 
     {"s3DataDestination":{"path": s3_output_path}}
)
batchInferenceJobArn = batchInferenceJobArn['batchInferenceJobArn']

다음 셀은 내보내기가 완료될 때까지 폴링합니다. 이는 이 워크숍의 마지막 대기 루프입니다. 여기서도 `ACTIVE` 상태가 될 때까지 폴링하며, 이 상태에 도달하면 계속 진행할 수 있습니다.

In [ ]:
current_time = datetime.datetime.now()
print("Import Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_inference_job_response = personalize_rec.describe_batch_inference_job(
        batchInferenceJobArn = batchInferenceJobArn
    )
    status = describe_dataset_inference_job_response["batchInferenceJob"]['status']
    print("DatasetInferenceJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.datetime.now()
print("Import Completed on: ", current_time.strftime("%I:%M:%S %p"))

데이터를 내보낸 후 파일을 가져와서 구문 분석합니다.

In [ ]:
s3 = boto3.client('s3')
export_name = json_input_filename + ".out"
s3.download_file(bucket_name, export_name, export_name)

# Update DF rendering
pd.set_option('display.max_rows', 30)
with open(export_name) as json_file:
    # Get the first line and parse it
    line = json.loads(json_file.readline())
    # Do the same for the other lines
    while line:
        # extract the user ID 
        col_header = "User: " + line['input']['userId']
        # Create a list for all the movies
        recommendation_list = []
        # Add all the entries
        for item in line['output']['recommendedItems']:
            title = get_movie_title(item)
            recommendation_list.append(title)
        if 'bulk_recommendations_df' in locals():
            new_rec_DF = pd.DataFrame(recommendation_list, columns = [col_header])
            bulk_recommendations_df = bulk_recommendations_df.join(new_rec_DF)
        else:
            bulk_recommendations_df = pd.DataFrame(recommendation_list, columns=[col_header])
        try:
            line = json.loads(json_file.readline())
        except:
            line = None
bulk_recommendations_df

위의 셀에서, 제공된 사용자에 대한 다양한 추천을 확인할 수 있습니다. 실제 시나리오에서는 사용자 목록이 조직의 전체 사용자 기반일 수 있으므로 이 방법으로 사용자 간의 결과를 신속하게 참조하고 비교할 수 있습니다.

## 추가 보너스: 정리

Event Engine을 사용하는 경우 워크숍을 완료하고 나면 계정이 삭제되므로, 아래의 셀은 전적으로 선택 사항입니다. 나중에 실제 계정에서 사용할 경우, 아래의 셀을 실행하면 생성된 리소스를 삭제하여 요금이 계속 발생하지 않도록 할 수 있습니다.

In [ ]:
# Delete the campaign:
personalize.delete_campaign(campaignArn=campaign_arn)
time.sleep(60)

In [ ]:
# Delete the solution
personalize.delete_solution(solutionArn=solution_arn)
time.sleep(60)

In [ ]:
# Delete the event tracker
personalize.delete_event_tracker(eventTrackerArn=event_tracker_arn)
time.sleep(60)

In [ ]:
# Delete the interaction dataset
personalize.delete_dataset(datasetArn=dataset_arn)
time.sleep(60)

In [ ]:
# Delete the event dataset
event_interactions_dataset_arn = dataset_arn
event_interactions_dataset_arn = event_interactions_dataset_arn.replace("INTERACTIONS", "EVENT_INTERACTIONS")
personalize.delete_dataset(datasetArn=event_interactions_dataset_arn)
time.sleep(60)

In [ ]:
# Delete the schema
personalize.delete_schema(schemaArn=schema_arn)

In [ ]:
# Delete the Dataset Group
personalize.delete_dataset_group(datasetGroupArn=dataset_group_arn)

In [ ]:
# Empty the S3 Bucket
s3 = boto3.resource('s3')
bucket = s3.Bucket(bucket_name)
bucket.objects.all().delete()
time.sleep(60)

In [ ]:
# Delete the S3 Bucket
bucket.delete()

In [ ]:
# IAM policies should also be removed
iam = boto3.client("iam")
iam.detach_role_policy(PolicyArn="arn:aws:iam::aws:policy/AmazonS3FullAccess", RoleName=role_name)
iam.detach_role_policy(PolicyArn="arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess",RoleName=role_name)

iam.delete_role(RoleName=role_name)

## 마지막 단계

리소스를 모두 정리하고 나면 이 창을 닫고 시작했던 Github 페이지로 돌아갈 수 있습니다. Readme 파일의 하단에 이전에 생성한 CloudFormation 스택을 삭제하는 단계가 나와 있습니다. 이 작업이 완료되면 워크샵을 100% 완료한 것입니다.

축하합니다!